In [ ]:
# Update this with your own stack name
stack_name = 'endpoint-one-model-2020-06-23-08-36-49'

In [ ]:
import boto3

cf = boto3.client('cloudformation')
response = cf.describe_stacks(StackName=stack_name)
endpoint_name = response['Stacks'][0]['Outputs'][0]['OutputValue']

print(endpoint_name)

In [ ]:
import os
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)
np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

In [ ]:
import random
import sagemaker
from sagemaker.tensorflow.serving import Predictor
from sagemaker.predictor import json_serializer

tf_predictor = Predictor(endpoint_name)
tf_predictor.content_type = 'application/json'
tf_predictor.serializer =  json_serializer

def predict_fmnist(num_samples=10):
    while True:
        indices = random.sample(range(x_val.shape[0] - 1), num_samples)
        images = x_val[indices]/255
        labels = y_val[indices]
        payload = images.reshape(num_samples, 28, 28, 1)
        response = tf_predictor.predict(payload)
        prediction = np.array(response['predictions'])
        #predicted_label = prediction.argmax(axis=1)
        #print('Predicted labels are: {}'.format(predicted_label))

In [ ]:
from threading import Thread
from time import sleep

thread = Thread(target=predict_fmnist)
thread.start()